# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

## Code

In [2]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [3]:
from datetime import datetime

### Cleaning and EDA

In [53]:
outage

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,CLIMATE.REGION,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,TOTAL.CUSTOMERS,OUTAGE.START,OUTAGE.RESTORATION,CUSTOMERS.PERCENTAGE
487,2012,1.0,Washington,WA,Northwest,cold,"Sunday, January 15, 2012",9:35:00 AM,"Sunday, January 15, 2012",9:35:00 AM,intentional attack,vandalism,NaN,0.0,NaN,NaN,3236484,2012-01-15 09:35:00,2012-01-15 09:35:00,NaN
314,2011,11.0,Indiana,IN,Central,cold,"Tuesday, November 22, 2011",11:16:00 PM,"Tuesday, November 22, 2011",11:16:00 PM,intentional attack,vandalism,NaN,0.0,0,0.0,3106397,2011-11-22 23:16:00,2011-11-22 23:16:00,0.000000
825,2011,8.0,Oregon,OR,Northwest,cold,"Monday, August 1, 2011",2:18:00 PM,"Monday, August 1, 2011",2:18:00 PM,intentional attack,vandalism,NaN,0.0,NaN,NaN,1895763,2011-08-01 14:18:00,2011-08-01 14:18:00,NaN
1400,2011,9.0,Connecticut,CT,Northeast,cold,"Friday, September 23, 2011",9:04:00 AM,"Friday, September 23, 2011",9:04:00 AM,intentional attack,vandalism,NaN,0.0,0,0.0,1608712,2011-09-23 09:04:00,2011-09-23 09:04:00,0.000000
410,2011,12.0,Washington,WA,Northwest,cold,"Saturday, December 31, 2011",9:26:00 PM,"Saturday, December 31, 2011",9:26:00 PM,intentional attack,vandalism,NaN,0.0,NaN,NaN,3219667,2011-12-31 21:26:00,2011-12-31 21:26:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,2002,NaN,Kansas,KS,South,NaN,NaN,NaN,NaN,NaN,severe weather,winter storm,NaN,NaN,550,270000.0,1379347,NaT,NaT,19.574480
1513,2015,6.0,Kansas,KS,South,warm,"Friday, June 26, 2015",2:00:00 AM,NaN,NaN,severe weather,NaN,NaN,NaN,NaN,110000.0,1484396,2015-06-26 02:00:00,NaT,7.410421
1527,2016,3.0,Idaho,ID,Northwest,warm,"Tuesday, March 1, 2016",1:35:00 PM,NaN,NaN,intentional attack,vandalism,NaN,NaN,0,0.0,849763,2016-03-01 13:35:00,NaT,0.000000
1530,2006,NaN,North Dakota,ND,West North Central,NaN,NaN,NaN,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,1650,NaN,366037,NaT,NaT,NaN


In [55]:
outage = pd.read_csv("outage.csv")
outage = outage.copy()
outage.columns = outage.iloc[4].values.tolist()
outage = outage[6:].reset_index().drop(columns="index")
outage = outage.drop(columns=["variables","OBS","ANOMALY.LEVEL","NERC.REGION"])
outage = outage.drop(outage.iloc[:, -22:], axis = 1)
outage = outage.drop(outage.loc[:, 'RES.PRICE':'IND.CUSTOMERS'].columns, axis = 1)
start_time = lambda row: pd.to_datetime(datetime.strptime(row['OUTAGE.START.DATE'] + ' ' + row['OUTAGE.START.TIME'], '%A, %B %d, %Y %I:%M:%S %p')) if (row['OUTAGE.START.DATE'] == row['OUTAGE.START.DATE']) & (row['OUTAGE.START.TIME'] == row['OUTAGE.START.TIME']) else np.NaN
end_time = lambda row: pd.to_datetime(datetime.strptime(row['OUTAGE.RESTORATION.DATE'] + ' ' + row['OUTAGE.RESTORATION.TIME'], '%A, %B %d, %Y %I:%M:%S %p')) if (row['OUTAGE.RESTORATION.DATE'] == row['OUTAGE.RESTORATION.DATE']) & (row['OUTAGE.RESTORATION.TIME'] == row['OUTAGE.RESTORATION.TIME']) else np.NaN
outage['OUTAGE.START'] = outage.apply(start_time, axis = 1)
outage['OUTAGE.RESTORATION'] = outage.apply(end_time, axis = 1)
outage["CUSTOMERS.PERCENTAGE"] = (pd.to_numeric(outage["CUSTOMERS.AFFECTED"],errors="coerce") / pd.to_numeric(outage["TOTAL.CUSTOMERS"],errors="coerce"))*100
outage.groupby("YEAR").count().max(axis=1)
outage["TOTAL.CUSTOMERS"] = pd.to_numeric(outage["TOTAL.CUSTOMERS"],errors="coerce")
outage["OUTAGE.DURATION"] = pd.to_numeric(outage["OUTAGE.DURATION"],errors="coerce")
new = outage[["OUTAGE.DURATION","POSTAL.CODE","U.S._STATE"]]
new = new[new["OUTAGE.DURATION"].isna()==False]
new = new.groupby(["U.S._STATE","POSTAL.CODE"]).median().reset_index()
chloro = px.choropleth(new,locations="POSTAL.CODE",color="OUTAGE.DURATION", locationmode="USA-states", 
                    scope="usa",
                    hover_name = "U.S._STATE",
                    color_continuous_scale = px.colors.sequential.Reds,title="Power Outage Duration in US")
chloro.write_html(r'C:\Users\Zly19\dsc80-2023-wi\projects\chloro.html', include_plotlyjs='cdn')
chloro

In [30]:
cause = px.histogram(outage, x='CAUSE.CATEGORY', nbins=50, histnorm='percent', 
                   title='Distribution of the Causes of Power Outage')
cause.write_html(r'C:\Users\Zly19\dsc80-2023-wi\projects\cause.html', include_plotlyjs='cdn')

In [35]:
# Univariate Analysis
#
state = px.histogram(outage, x="POSTAL.CODE", nbins=50, histnorm='percent', 
                   title='State Distribution of Power Outages in the US')
state.write_html(r'C:\Users\Zly19\dsc80-2023-wi\projects\state.html', include_plotlyjs='cdn')
state

In [31]:
outage["CLIMATE.REGION.FIXED"] = outage['CLIMATE.REGION'].fillna('unknown')
climate = px.histogram(outage, x="CLIMATE.REGION.FIXED", nbins=50, histnorm='percent', 
                   title='Distribution of the Climate Regions of Power Outage')
climate
# climate.write_html(r'C:\Users\Zly19\dsc80-2023-wi\projects\climate.html', include_plotlyjs='cdn')

In [45]:
outage["MONTH"] = pd.to_numeric(outage["MONTH"],errors="coerce")
mon_cause = px.box(outage, x='CAUSE.CATEGORY', y="MONTH",title="Distribution of Months for Different Causes")
mon_cause.write_html(r'C:\Users\Zly19\dsc80-2023-wi\projects\mon_cause.html', include_plotlyjs='cdn')
mon_cause

In [51]:
outage["YEAR"] = outage["YEAR"].astype(int)
outage["CUSTOMERS.AFFECTED"] = pd.to_numeric(outage["CUSTOMERS.AFFECTED"],errors="coerce")
fig = px.box(outage, x='YEAR', y="CUSTOMERS.AFFECTED", title="Box plot of months that outages occur for each cause")
fig.show()

In [48]:
# Bivariate Analysis
grouped_bar = outage.copy()

group = {}
group['CLIMATE'] = ['normal', 'cold', 'warm']
# Collect the value counts of each climate category for each causes
for category in grouped_bar["CAUSE.CATEGORY"].unique():
    result = grouped_bar[grouped_bar["CAUSE.CATEGORY"] == category]['CLIMATE.CATEGORY'].value_counts().to_list()
    group[category] = result

df = pd.DataFrame(group)
# Calculating Distribution of Climates Category Among Different Causes
df.iloc[:,1:] = df.iloc[:,1:]/ df.iloc[:,1:].sum()

# melt the dataframe to long format
df_melt = pd.melt(df, id_vars=['CLIMATE'], var_name='Category', value_name='Value')

# create grouped bar chart
fig = px.bar(df_melt, x='Category', y='Value', color='CLIMATE', barmode='group')

fig.update_layout(
    title="Distribution of Climates Category Among Different Causes",
    xaxis_title="Causes",
    yaxis_title="Percentage",
    legend_title="climate category",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="RebeccaPurple"
    )
)
fig.write_html(r'C:\Users\Zly19\dsc80-2023-wi\projects\climate_cause.html', include_plotlyjs='cdn')
fig

In [52]:
fig = px.box(outage, x='YEAR', y="TOTAL.CUSTOMERS", title="Box plot of months that outages occur for each cause")
fig.show()

### Assessment of Missingness

In [167]:
outage['affected_customer_missing'] = outage['CUSTOMERS.AFFECTED'].isna()
observed_dist = outage.pivot_table(index = 'CAUSE.CATEGORY', columns = 'affected_customer_missing', aggfunc='size').apply(lambda x: x/x.sum())
observed_tvd = observed_dist.diff(axis = 1).iloc[:, -1].abs().sum() / 2
observed_tvd

0.5574337954907068

In [168]:
#permutation test
# null hypothesis: the distribution of causes of outages when affected_customers is missing 
#is the same as the distribution of causes of outages when affected_customers is not missing
n_repetitions = 500
shuffled = outage.copy()

tvds = []
for _ in range(n_repetitions):
    shuffled['CAUSE.CATEGORY'] = np.random.permutation(shuffled['CAUSE.CATEGORY'])
    
    pivoted = (
        shuffled
        .pivot_table(index = 'CAUSE.CATEGORY', columns = 'affected_customer_missing', aggfunc='size')
        .apply(lambda x: x/x.sum())
    )
    
    tvd = pivoted.diff(axis = 1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)
np.mean(np.array(tvds) >= observed_tvd)
#reject the null
#Hence, we conclude that the missingness in the affected_customers column is dependent on causes of power outages

0.0

In [169]:
outage['TOTAL.CUSTOMERS'] = outage['TOTAL.CUSTOMERS'].astype(int)
observed_mean = outage.groupby('affected_customer_missing')['TOTAL.CUSTOMERS'].mean().diff().iloc[-1]
observed_mean

59563.63563984446

In [170]:
#permutation test
# null hypothesis: the distribution of total customers when affected_customers is missing 
#is the same as the distribution of total customers when affected_customers is not missing
n_repetitions = 500
shuffled = outage.copy()

diff_means = []
for _ in range(n_repetitions):
    # Shuffling the values
    shuffled['TOTAL.CUSTOMERS'] = np.random.permutation(shuffled['TOTAL.CUSTOMERS'])
    
    # Computing and storing the absolute difference in means
    diff_mean = shuffled.groupby('affected_customer_missing')['TOTAL.CUSTOMERS'].mean().diff().abs().iloc[-1]
    
    tvd = pivoted.diff(axis = 1).iloc[:, -1].abs().sum() / 2
    diff_means.append(diff_mean)
np.mean(np.array(diff_means) >= observed_mean)
#fail to reject the null
#Hence, we conclude that the missingness in the affected_customers column is not dependent on total customers

0.788

In [173]:
outage["MONTH"] = pd.to_numeric(outage["MONTH"],errors="coerce")
outage["CUSTOMERS.AFFECTED"] = pd.to_numeric(outage["CUSTOMERS.AFFECTED"],errors="coerce")
outage["YEAR"] = outage["YEAR"].astype(int)

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,CLIMATE.REGION,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,...,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,TOTAL.CUSTOMERS,OUTAGE.START,OUTAGE.RESTORATION,CUSTOMERS.PERCENTAGE,affected_customer_missing
0,2011,7.0,Minnesota,MN,East North Central,normal,"Friday, July 1, 2011",5:00:00 PM,"Sunday, July 3, 2011",8:00:00 PM,...,NaN,NaN,3060.0,NaN,70000.0,2595696,2011-07-01 17:00:00,2011-07-03 20:00:00,2.696772,False
1,2014,5.0,Minnesota,MN,East North Central,normal,"Sunday, May 11, 2014",6:38:00 PM,"Sunday, May 11, 2014",6:39:00 PM,...,vandalism,NaN,1.0,NaN,NaN,2640737,2014-05-11 18:38:00,2014-05-11 18:39:00,NaN,True
2,2010,10.0,Minnesota,MN,East North Central,cold,"Tuesday, October 26, 2010",8:00:00 PM,"Thursday, October 28, 2010",10:00:00 PM,...,heavy wind,NaN,3000.0,NaN,70000.0,2586905,2010-10-26 20:00:00,2010-10-28 22:00:00,2.705936,False
3,2012,6.0,Minnesota,MN,East North Central,normal,"Tuesday, June 19, 2012",4:30:00 AM,"Wednesday, June 20, 2012",11:00:00 PM,...,thunderstorm,NaN,2550.0,NaN,68200.0,2606813,2012-06-19 04:30:00,2012-06-20 23:00:00,2.616221,False
4,2015,7.0,Minnesota,MN,East North Central,warm,"Saturday, July 18, 2015",2:00:00 AM,"Sunday, July 19, 2015",7:00:00 AM,...,NaN,NaN,1740.0,250,250000.0,2673531,2015-07-18 02:00:00,2015-07-19 07:00:00,9.350930,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011,12.0,North Dakota,ND,West North Central,cold,"Tuesday, December 6, 2011",8:00:00 AM,"Tuesday, December 6, 2011",8:00:00 PM,...,NaN,NaN,720.0,155,34500.0,394394,2011-12-06 08:00:00,2011-12-06 20:00:00,8.747598,False
1530,2006,NaN,North Dakota,ND,West North Central,NaN,NaN,NaN,NaN,NaN,...,Coal,NaN,NaN,1650,NaN,366037,NaT,NaT,NaN,True
1531,2009,8.0,South Dakota,SD,West North Central,warm,"Saturday, August 29, 2009",10:54:00 PM,"Saturday, August 29, 2009",11:53:00 PM,...,NaN,NaN,59.0,84,NaN,436229,2009-08-29 22:54:00,2009-08-29 23:53:00,NaN,True
1532,2009,8.0,South Dakota,SD,West North Central,warm,"Saturday, August 29, 2009",11:00:00 AM,"Saturday, August 29, 2009",2:01:00 PM,...,NaN,NaN,181.0,373,NaN,436229,2009-08-29 11:00:00,2009-08-29 14:01:00,NaN,True


In [ ]:
# TODO

### Hypothesis Testing

In [ ]:
# TODO